# MathLang Learning Log Demo
`docs/test/TestPlan.md` の P3-01 / P3-02 シナリオを参考に、PolynomialEvaluator と LearningLogger の連携を可視化します。

## シナリオの焦点
- `TC-P301-03`: Notebook 上での多項式演習シナリオ
- `TC-P301-04`: LearningLogger を用いた学習ログの可視化
（`TC-P301-02` Hello World シナリオは CLI の自己診断でカバーされるため、本ノートでは多項式とログにフォーカスします。）

In [ ]:
from pathlib import Path
import sys

repo_root = Path.cwd().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
repo_root

## ヘルパーセットアップ
Core DSL を解析して PolynomialEvaluator / LearningLogger を同時に動かす関数を用意します。

In [ ]:
from textwrap import dedent
from core.logging import LearningLogger
from core.parser import Parser
from core.polynomial_evaluator import PolynomialEvaluator
from core.i18n import get_language_pack


def run_polynomial_demo(source: str, language_code: str = "en"):
    """Compile the DSL, run the evaluator, and capture the learning log."""
    language = get_language_pack(language_code)
    program = Parser(dedent(source), language=language).parse()
    logger = LearningLogger()
    evaluator = PolynomialEvaluator(program, language=language, learning_logger=logger)
    results = evaluator.run()
    return evaluator, results, logger

## デモ用 DSL スクリプト
`docs/test/TestPlan.md` の `TC-P205-04/05/06` と `TC-P301-03` を踏まえて、分配法則・二項展開・定数除算・結合法則の確認を 1 つの Core DSL で表現します。

In [ ]:
demo_source = """
# Distributive / expansion scenarios inspired by the TestPlan
problem: (x + 1) + (x + 2)
step1: x + 1 + x + 2
step2: 2*x + 3
end: 2*x + 3

problem: x * (y + z)
step1: x*y + x*z
end: x*y + x*z

problem: (x + y)^2
step1: x^2 + 2*x*y + y^2
end: x^2 + 2*x*y + y^2

normalized = (2 * x) / 2
show normalized

left = (x + y) + z
right = x + (y + z)
show left
show right
"""
demo_source

## PolynomialEvaluator を実行
上記の DSL を実行し、各ステップメッセージ（`problem`/`step`/`end`/`show`）を確認します。

In [ ]:
evaluator, results, logger = run_polynomial_demo(demo_source)
for res in results:
    prefix = f"step {res.step_number:02d}" if res.step_number else "info"
    print(f"{prefix}: {res.message}")

## 文脈の検証
`TC-P205-03` (結合法則) や `TC-P205-06` (定数除算の正規化) が満たされているか、Evaluator の文脈を直接チェックします。

In [ ]:
left = evaluator.context['left']
right = evaluator.context['right']
normalized = evaluator.context['normalized']

print('Associativity check (left == right):', left == right, '→', left)
print('Scalar division normalized:', normalized)

## LearningLogger の確認
`TC-P301-04` の通り、PolynomialEvaluator から送られた学習ログを JSON として可視化します。`rule_id` によってどのナレッジノードが検証に使われたかが分かります。

In [ ]:
from IPython.display import JSON
JSON(logger.to_dict())